In [3]:
import torchaudio

In [4]:
import pandas as pd

file_path = '../Data/240515 Subject 5 labelled.xlsx'
df = pd.read_excel(file_path)
df_breath = df[df["Label phase 1"] == "Breathing"]
breathing_indexes = df_breath.index
print(df_breath.head())

        Time Label phase 1 Breathing Cadence Label Phase 2
18  00:00:38     Breathing                 x     Breathing
19  00:00:40     Breathing                 x     Breathing
20  00:00:42     Breathing                 x     Breathing
21  00:00:44     Breathing                 x     Breathing
22  00:00:46     Breathing                 x     Breathing


In [5]:
breathing_indexes = df_breath.index

In [7]:
waveform, sr = torchaudio.load("../Data/converted_mp3_240515 Subject 5.wav") # this returns a tuple of the waveform and the sample rate
waveform_out = waveform[0]
GROUPBY = 2 * sr # 1 seconds
spec_out = waveform_out[:waveform_out.shape[0] // GROUPBY * GROUPBY].T
wf_grouped = spec_out.reshape(-1, GROUPBY)
wf_grouped.shape

/var/folders/pl/7f1cr2657p3bnpdt3bnm_9w00000gn/T/ipykernel_11674/2945954212.py:4: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3679.)
  spec_out = waveform_out[:waveform_out.shape[0] // GROUPBY * GROUPBY].T


torch.Size([14435, 96000])

In [8]:
result = []
current_group = [breathing_indexes[0]]

for idx in breathing_indexes[1:]:
    if idx == current_group[-1] + 1:
        current_group.append(idx)
    else:
        result.append(current_group)
        current_group = [idx]

result.append(current_group)

In [56]:
result[-3:]

[[799, 800], [832], [837]]

In [10]:
import numpy as np
from tqdm import tqdm

breathing_files = []
for lst in tqdm(result):
    temp_list = []
    
    for v in lst:
        temp_list.append(wf_grouped[v])
    
    temp = np.concatenate(temp_list)
    breathing_files.append(temp)

100%|██████████| 41/41 [00:00<00:00, 177.85it/s]


In [45]:
def breathing_map(var):
    if var < 300:
        return "Stable"
    elif var < 500: 
        return "Slightly unstable"
    else:
        return "Dangerous"

In [51]:
def find_middle_indices(mask, DISTANCE, DISTANCE_NEXT):
    saved_indices = []
    n = len(mask)
    
    for i in range(DISTANCE, n - DISTANCE):
        if all(mask[i - DISTANCE:i]) and all(mask[i + 1:i + 1 + DISTANCE]):
            # Check if this index is far enough from the last saved index
            if not saved_indices or (i - saved_indices[-1]) >= DISTANCE_NEXT:
                saved_indices.append(i)
    
    return np.array(saved_indices)

In [54]:
from scipy.signal import butter, filtfilt, savgol_filter, argrelextrema, find_peaks
import numpy as np
import librosa
import matplotlib.pyplot as plt

for signal in breathing_files:
    # plt.plot(np.linspace(0, len(signal)/sr, len(signal)), signal)

    lowcut = 1000
    highcut = 10000
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    min_25_procent = np.mean(np.abs(sorted(filtered_signal)[:len(filtered_signal)//4]))
    top_25_procent = np.mean(np.abs(sorted(filtered_signal)[len(filtered_signal)//4:]))
    mask = abs(filtered_signal) > top_25_procent



    # plt.plot(np.linspace(0, len(signal)/sr, len(signal)), filtered_signal)
    # plt.axhline(y=low, color='r', linestyle='--', label=f'Low cut: {lowcut} Hz')
    # plt.axhline(y=high, color='g', linestyle='--', label=f'High cut: {highcut} Hz')

    # Generate a mask where the distance condition is fulfilled
    # Apply the mask to the extrema search
    peaks = find_middle_indices(mask, 20, sr)
    if len(peaks) == 0:
        continue

    # print(peaks/sr)

    # plt.scatter(peaks/sr, [0.004 for x in peaks])

    breath_durations = np.diff(peaks) / sr

    avg_breath_duration = np.mean(breath_durations)
    breathing_rate = 60 / avg_breath_duration  # Convert breath duration to breaths per minute

    breathing_rate_per_minute = 60/breath_durations
    breathing_rate_variance = np.var(breathing_rate_per_minute)
    breathing_rate_variance_string = breathing_map(breathing_rate_variance)
    if np.isnan(breathing_rate):
        print("Could not get breathing rate.")
    else:
        print('Estimated breathing rate: ', breathing_rate, ' breaths per minute | With variance: ', breathing_rate_variance_string)

Estimated breathing rate:  27.102403969544014  breaths per minute | With variance:  Stable
Estimated breathing rate:  26.90552796493313  breaths per minute | With variance:  Stable
Could not get breathing rate.
Estimated breathing rate:  20.35122828142357  breaths per minute | With variance:  Stable
Could not get breathing rate.
Estimated breathing rate:  20.390776005909547  breaths per minute | With variance:  Stable
Could not get breathing rate.
Estimated breathing rate:  20.47628350037462  breaths per minute | With variance:  Stable
Estimated breathing rate:  18.478990841777915  breaths per minute | With variance:  Stable
Estimated breathing rate:  19.31081436551287  breaths per minute | With variance:  Stable
Estimated breathing rate:  19.663851913716115  breaths per minute | With variance:  Stable
Estimated breathing rate:  22.566288472387637  breaths per minute | With variance:  Stable
Estimated breathing rate:  58.80071050858531  breaths per minute | With variance:  Stable
Estim